In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
# Load the dataset and preprocess as before
df = pd.read_csv('D:/ML_Assignments/home_data.csv')

C:\Users\keert\AppData\Local\Temp\ipykernel_6492\4128364433.py:2: DtypeWarning: Columns (0,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/ML_Assignments/home_data.csv')


In [5]:
# Convert 'time' column to datetime format and coerce errors
df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [7]:
# Define a single target column and weather-related input columns
target_column = ['House overall [kW]']
weather_columns = ['temperature', 'humidity', 'visibility', 'apparentTemperature',
                   'pressure', 'windSpeed', 'windBearing', 'dewPoint']

In [9]:
# Drop rows with missing values in the target column and fill weather-related columns
df_cleaned = df.dropna(subset=target_column)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')

C:\Users\keert\AppData\Local\Temp\ipykernel_6492\1058757868.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Local\Temp\ipykernel_6492\1058757868.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')


In [10]:
# Separate input features and target variable
X = df_cleaned[weather_columns]
y = df_cleaned[target_column]

In [11]:
# Scale features and target column
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [12]:
# Function to create sequences
def create_sequences(X, y, time_steps=30):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [13]:
# Generate sequences
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42, shuffle=False)

In [10]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(56, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(28, activation='tanh', return_sequences=True))
model.add(LSTM(14, activation='tanh'))
model.add(Dense(1))  # Single output

C:\Users\keert\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [11]:
# Compile the model with MAE as the loss function
model.compile(optimizer='adam', loss='mean_absolute_error')

In [12]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00005, restore_best_weights=True)

In [13]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=50,
    batch_size=128,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 140s 67ms/step - loss: 0.0285 - val_loss: 0.0625
Epoch 2/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 128s 66ms/step - loss: 0.0270 - val_loss: 0.0612
Epoch 3/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 125s 65ms/step - loss: 0.0266 - val_loss: 0.0581
Epoch 4/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 118s 61ms/step - loss: 0.0264 - val_loss: 0.0579
Epoch 5/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 94s 49ms/step - loss: 0.0261 - val_loss: 0.0588
Epoch 6/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 111s 57ms/step - loss: 0.0258 - val_loss: 0.0575
Epoch 7/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 129s 67ms/step - loss: 0.0256 - val_loss: 0.0610
Epoch 8/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 118s 61ms/step - loss: 0.0253 - val_loss: 0.0590
Epoch 9/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 124s 64ms/step - loss: 0.0250 - val_loss: 0.0593
Epoch 10/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 127s 66ms/step - loss: 0.0248 - val_loss: 0.0597
Epoch 11/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 126s 65ms/step - loss: 0.0242 - val_loss: 0.060

In [14]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss (MAE): {loss}')

Test Loss (MAE): 0.029298264533281326


In [15]:
# Make predictions
y_pred_scaled = model.predict(X_test)

4724/4724 ━━━━━━━━━━━━━━━━━━━━ 66s 14ms/step


In [16]:
# Inverse transform the predictions and the actual values
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test)

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Load the dataset and preprocess as before
df = pd.read_csv('D:/ML_Assignments/home_data.csv')

C:\Users\keert\AppData\Local\Temp\ipykernel_16180\4128364433.py:2: DtypeWarning: Columns (0,27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('D:/ML_Assignments/home_data.csv')


In [3]:
# Convert 'time' column to datetime format and coerce errors
df['time'] = pd.to_datetime(df['time'], errors='coerce')

In [4]:
# Define a single target column and weather-related input columns
target_column = ['gen [kW]']
weather_columns = ['temperature', 'humidity', 'visibility', 'apparentTemperature',
                   'pressure', 'windSpeed', 'windBearing', 'dewPoint']


# Drop rows with missing values in the target column and fill weather-related columns
df_cleaned = df.dropna(subset=target_column)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')


C:\Users\keert\AppData\Local\Temp\ipykernel_16180\881302698.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Local\Temp\ipykernel_16180\881302698.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')


In [5]:
# Separate input features and target variable
X = df_cleaned[weather_columns]
y = df_cleaned[target_column]

# Scale features and target column
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [6]:
# Function to create sequences
def create_sequences(X, y, time_steps=30):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i + time_steps])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [7]:
# Generate sequences
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42, shuffle=False)

In [8]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(56, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(28, activation='tanh', return_sequences=True))
model.add(LSTM(14, activation='tanh'))
model.add(Dense(1))  # Single output

C:\Users\keert\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Compile the model with MAE as the loss function
model.compile(optimizer='adam', loss='mean_absolute_error')

In [10]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00005, restore_best_weights=True)

In [11]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=50,
    batch_size=128,
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 134s 65ms/step - loss: 0.1240 - val_loss: 0.1504
Epoch 2/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 134s 70ms/step - loss: 0.1227 - val_loss: 0.1498
Epoch 3/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 116s 60ms/step - loss: 0.1234 - val_loss: 0.1493
Epoch 4/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 173s 90ms/step - loss: 0.1230 - val_loss: 0.1494
Epoch 5/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 131s 68ms/step - loss: 0.1207 - val_loss: 0.1524
Epoch 6/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 116s 60ms/step - loss: 0.1219 - val_loss: 0.1492
Epoch 7/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 111s 58ms/step - loss: 0.1233 - val_loss: 0.1497
Epoch 8/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 116s 60ms/step - loss: 0.1233 - val_loss: 0.1504
Epoch 9/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 116s 60ms/step - loss: 0.1224 - val_loss: 0.1494
Epoch 10/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 116s 60ms/step - loss: 0.1236 - val_loss: 0.1494
Epoch 11/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 115s 60ms/step - loss: 0.1232 - val_loss: 0.15

In [12]:
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss (MAE): {loss}')

Test Loss (MAE): 0.09416600316762924


In [13]:
# Define the target column 'Kitchen 12 [kW]'
critical_columns = ['Kitchen 12 [kW]']

# Drop rows with missing values in critical target column and forward-fill/backward-fill weather columns
df_cleaned = df.dropna(subset=critical_columns)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')

# Separate input features and target variables
X = df_cleaned[weather_columns]
y = df_cleaned[critical_columns]

# Scale features and target columns
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Generate sequences
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42, shuffle=False)

# Build the LSTM model
model = Sequential()
model.add(LSTM(56, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(28, activation='tanh', return_sequences=True))
model.add(LSTM(14, activation='tanh'))
model.add(Dense(1))  # Single output for 'Kitchen 12 [kW]'

# Compile the model with MAE as the loss function
model.compile(optimizer='adam', loss='mean_absolute_error')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00005, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=50,
    batch_size=128,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss (MAE): {loss}')

# Make predictions
y_pred_scaled = model.predict(X_test)

# Inverse transform the predictions and the actual values
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test)

# Print the first few predictions and actual values for comparison
print(f'Predictions: {y_pred[:5]}')
print(f'Actual Values: {y_test_original[:5]}')


C:\Users\keert\AppData\Local\Temp\ipykernel_16180\624685616.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Local\Temp\ipykernel_16180\624685616.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` o

Epoch 1/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 128s 62ms/step - loss: 0.0041 - val_loss: 0.0027
Epoch 2/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 117s 60ms/step - loss: 0.0011 - val_loss: 0.0025
Epoch 3/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 114s 59ms/step - loss: 9.7192e-04 - val_loss: 0.0024
Epoch 4/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 110s 57ms/step - loss: 9.5900e-04 - val_loss: 0.0024
Epoch 5/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 129s 67ms/step - loss: 8.6741e-04 - val_loss: 0.0024
Epoch 6/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 132s 69ms/step - loss: 9.0329e-04 - val_loss: 0.0027
Epoch 7/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 128s 67ms/step - loss: 8.9523e-04 - val_loss: 0.0024
Epoch 8/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 129s 67ms/step - loss: 8.6701e-04 - val_loss: 0.0025
Epoch 9/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 137s 71ms/step - loss: 8.8030e-04 - val_loss: 0.0024
Epoch 10/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 138s 71ms/step - loss: 8.6134e-04 - val_loss: 0.0025
Epoch 11/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 142s 73ms/step

In [27]:
# Define the target column 'Dishwasher [kW]'
critical_columns = ['Dishwasher [kW]']

# Drop rows with missing values in critical target column and forward-fill/backward-fill weather columns
df_cleaned = df.dropna(subset=critical_columns)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')

# Separate input features and target variables
X = df_cleaned[weather_columns]
y = df_cleaned[critical_columns]

# Scale features and target columns
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Generate sequences
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42, shuffle=False)

# Build the LSTM model
model = Sequential()
model.add(LSTM(56, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(28, activation='tanh', return_sequences=True))
model.add(LSTM(14, activation='tanh'))
model.add(Dense(1))  # Single output for 'Dishwasher [kW]'

# Compile the model with MAE as the loss function
model.compile(optimizer='adam', loss='mean_absolute_error')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00005, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=50,
    batch_size=128,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss (MAE): {loss}')

# Make predictions
y_pred_scaled = model.predict(X_test)

# Inverse transform the predictions and the actual values
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test)

# Print the first few predictions and actual values for comparison
print(f'Predictions: {y_pred[:5]}')
print(f'Actual Values: {y_test_original[:5]}')


C:\Users\keert\AppData\Local\Temp\ipykernel_16180\3242260468.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Local\Temp\ipykernel_16180\3242260468.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)`

Epoch 1/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 165s 77ms/step - loss: 0.0262 - val_loss: 0.0210
Epoch 2/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 147s 76ms/step - loss: 0.0235 - val_loss: 0.0203
Epoch 3/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 154s 80ms/step - loss: 0.0231 - val_loss: 0.0201
Epoch 4/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 161s 84ms/step - loss: 0.0228 - val_loss: 0.0201
Epoch 5/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 196s 102ms/step - loss: 0.0229 - val_loss: 0.0202
Epoch 6/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 170s 88ms/step - loss: 0.0233 - val_loss: 0.0201
Epoch 7/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 170s 88ms/step - loss: 0.0229 - val_loss: 0.0201
Epoch 8/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 182s 94ms/step - loss: 0.0230 - val_loss: 0.0201
Epoch 9/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 162s 84ms/step - loss: 0.0232 - val_loss: 0.0203
Epoch 10/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 157s 81ms/step - loss: 0.0228 - val_loss: 0.0202
Epoch 11/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 145s 75ms/step - loss: 0.0230 - val_loss: 0.0

In [14]:
# Define the target column 'Living room [kW]'
critical_columns = ['Living room [kW]']

# Drop rows with missing values in critical target column and forward-fill/backward-fill weather columns
df_cleaned = df.dropna(subset=critical_columns)
df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')

# Separate input features and target variables
X = df_cleaned[weather_columns]
y = df_cleaned[critical_columns]

# Scale features and target columns
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

# Generate sequences
X_sequences, y_sequences = create_sequences(X_scaled, y_scaled, time_steps=60)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.3, random_state=42, shuffle=False)

# Build the LSTM model
model = Sequential()
model.add(LSTM(56, activation='tanh', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(28, activation='tanh', return_sequences=True))
model.add(LSTM(14, activation='tanh'))
model.add(Dense(1))  # Single output for 'Living room [kW]'

# Compile the model with MAE as the loss function
model.compile(optimizer='adam', loss='mean_absolute_error')

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, min_delta=0.00005, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_split=0.3,
    epochs=50,
    batch_size=128,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss (MAE): {loss}')

# Make predictions
y_pred_scaled = model.predict(X_test)

# Inverse transform the predictions and the actual values
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_original = scaler_y.inverse_transform(y_test)

# Print the first few predictions and actual values for comparison
print(f'Predictions: {y_pred[:5]}')
print(f'Actual Values: {y_test_original[:5]}')


C:\Users\keert\AppData\Local\Temp\ipykernel_6492\461743820.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Local\Temp\ipykernel_6492\461743820.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[weather_columns] = df_cleaned[weather_columns].fillna(method='ffill').fillna(method='bfill')
C:\Users\keert\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` obj

Epoch 1/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 99s 48ms/step - loss: 0.0740 - val_loss: 0.0623
Epoch 2/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 99s 51ms/step - loss: 0.0708 - val_loss: 0.0626
Epoch 3/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 99s 51ms/step - loss: 0.0708 - val_loss: 0.0623
Epoch 4/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 102s 53ms/step - loss: 0.0705 - val_loss: 0.0625
Epoch 5/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 92s 48ms/step - loss: 0.0716 - val_loss: 0.0627
Epoch 6/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 79s 41ms/step - loss: 0.0718 - val_loss: 0.0629
Epoch 7/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 79s 41ms/step - loss: 0.0715 - val_loss: 0.0627
Epoch 8/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 88s 46ms/step - loss: 0.0714 - val_loss: 0.0628
Epoch 9/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 91s 47ms/step - loss: 0.0710 - val_loss: 0.0626
Epoch 10/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 89s 46ms/step - loss: 0.0706 - val_loss: 0.0626
Epoch 11/50
1929/1929 ━━━━━━━━━━━━━━━━━━━━ 93s 48ms/step - loss: 0.0716 - val_loss: 0.0627
Test Lo